In [9]:
from Narratives import *

In [ ]:
narratives_init(
    n_tenant_id = '<your-tenant-id>',
    n_client_id = '<your-client-id>',
    n_client_secret = '<your-clinet-secret>',
    n_sub_key = '<api-key>',

    n_deployment_id = "gpt-4o",
    n_api_version = "2024-06-01",
    
    n_workspace_name = "EOU 360 Reporting",
    n_dataset_name = 'Volume Monthly',
    n_timestamp = datetime.datetime.now()
)

## Get Current Month
q = f"""
EVALUATE
VAR MaxMonth445 = 
    CALCULATE(
        MAX('NSR Data'[Month 445]),
        'NSR Data'[Closed Month 445] = "Y"
    )
RETURN  SUMMARIZECOLUMNS(
	'NSR Data'[Month 445],
	FILTER('NSR Data',[Month 445] = MaxMonth445),
	"MonthNumber", MONTH(MaxMonth445),
	"FormatMonth", FORMAT(MaxMonth445,"yyyy_mmm")
)"""
LatestClosedMonth = PowerBI_Query(q)["[MonthNumber]"][0]
CurrentYear = datetime.datetime.now().year

Access Token: eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6ImltaTBZMnowZFlLeEJ0dEFxS19UdDVoWUJUayIsImtpZCI6ImltaTBZMnowZFlLeEJ0dEFxS19UdDVoWUJUayJ9.eyJhdWQiOiJodHRwczovL2FuYWx5c2lzLndpbmRvd3MubmV0L3Bvd2VyYmkvYXBpIiwiaXNzIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvNTQ4ZDI2YWItOGNhYS00OWUxLTk3YzItYTFiMWEwNmNjMzljLyIsImlhdCI6MTczOTU0NTc2NCwibmJmIjoxNzM5NTQ1NzY0LCJleHAiOjE3Mzk1NDk2NjQsImFpbyI6ImsyUmdZSmkvMTB0M2hzOExTejNyTFA0L1o4WHZBZ0E9IiwiYXBwaWQiOiI4OTNkNTA1My0zZDQwLTQ1ZjAtOThmMy02OWY4NzljMGVjY2UiLCJhcHBpZGFjciI6IjEiLCJpZHAiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC81NDhkMjZhYi04Y2FhLTQ5ZTEtOTdjMi1hMWIxYTA2Y2MzOWMvIiwiaWR0eXAiOiJhcHAiLCJvaWQiOiI2N2MyYTdjYi0wOTEwLTQ1MmQtOTE0Yy0wNzEzMGJlMTFjNTQiLCJyaCI6IjEuQVNZQXF5YU5WS3FNNFVtWHdxR3hvR3pEbkFrQUFBQUFBQUFBd0FBQUFBQUFBQUFtQUFBbUFBLiIsInN1YiI6IjY3YzJhN2NiLTA5MTAtNDUyZC05MTRjLTA3MTMwYmUxMWM1NCIsInRpZCI6IjU0OGQyNmFiLThjYWEtNDllMS05N2MyLWExYjFhMDZjYzM5YyIsInV0aSI6IjFOay1zQmJIaVVLSGZLNGpKLU5SQUEiLCJ2ZXIiOiIxLjAiLCJ4bXNfaWRyZWwiOiIyNCA3In0.KIVi9i4svYLOR9aIFTppu4RdM

In [ ]:
drill_metric = "[Unit Cases vs PY]"
prompt = """
Produce a formal narrative of this data. Do not include anything other than the narrative in your response. 
Instructions: Discuss the metrics provided for each of the keys. 'Absolute Unit Cases Growth' is the key metric to discuss as it represents absolute growth from prior year. 
'Unit Cases % Growth' is the relative percentage increase from prior year. 
'Last Year Unit Cases % Growth' is provided also as a reference to compare how the growth has changed from last year's progress. 
'vs Target' references unit cases against the business plan. 
Talk about the metrics in the order of 'Unit Cases % Growth', 'Last Year Unit Cases % Growth', and then mention the difference vs target which is 'Unit Cases % vs Target' and 'Absolute Unit Cases vs Target'. 
Avoid using overly superfluous words (significant, modest, notable) to describe performance.Rather, focus on the simplicity of the narrative and then the dissemination. 
Avoid all adjectives or descriptive words for the data.
"""
summary_prompt="""
Rewrite this long narrative to build a report out of our performance this month. 
Currently each subsection was written without context of it's parent section. 
I want a new narrative that can look at all of these sections holistically and produce an insightful description of Europe's performance. 
The hierarchy follows Europe to Zone to Country, with a breakdown of Category and Channel at each of these levels. 
So if you see certain channels or categories are performing at each level of the drill down, note this as a single insight rather than repeating it. 
The narrative should be 'Category X is doing well everywhere'. Do not repeat the same insights as I explained in the context of channel and category. 
Write me a narrative in the context of reporting out the monthly performance for the company. Keep it factual and insightful.
"""
filters = f"""
FILTER(
	'NSR Data',
	[Year 445] = {CurrentYear} && 
	[Closed Month 445] = "Y" && 
	MONTH('NSR Data'[Month 445]) <= {LatestClosedMonth} &&
	[L4 - Operations] <> "Unassigned" &&
	[Global Ventures Indicator] = "Core Brands (Non GV)" && 
	[Beverage Category] <> "Unassigned" &&
    [L3 - Market Type] IN {{"Home", "AFH"}}
)"""
metrics = """
"Absolute Unit Cases Growth",
			IF(
				CALCULATE([Unit Cases vs PY]) < 0,
				"-" & FORMAT(
					ROUND(
						ABS(CALCULATE([Unit Cases vs PY])) / 1000000, 1
					), "0.0") & " M",
				FORMAT(
					ROUND(
						CALCULATE([Unit Cases vs PY]) / 1000000, 1
					), "0.0") & " M"
			),
    "Unit Cases % Growth",
        FORMAT(
            ROUND(
                CALCULATE([Unit Cases % vs PY (CD)]) * 100, 1
            ), "0.0") & " %",
    "Last year Unit Cases % Growth", 
        FORMAT(
            ROUND(
                CALCULATE([Unit Cases Cycling % (CD)]) * 100, 1
            ), "0.0") & " %",
    "Absolute Unit Cases vs Target", 
			IF(
				CALCULATE([Unit Cases vs BP]) < 0,
				"-" & FORMAT(
					ROUND(
						ABS(CALCULATE([Unit Cases vs BP])) / 1000000, 1
					), "0.0") & " M",
				FORMAT(
					ROUND(
						CALCULATE([Unit Cases vs BP]) / 1000000, 1
					), "0.0") & " M"
			)
"""
query_json = {
    'filters': filters,
    'metrics':metrics,
    'drill_metric': drill_metric,
    'dimensions': []
}

In [13]:
data = []
##Europe
query_json['dimensions'] = ["'NSR Data'[L5 - Operating Unit]"]
data.append(single_level(query_json))

##Channel
query_json['dimensions'] = ["'NSR Data'[L3 - Market Type]"]
data.append(single_level(query_json))

##Category
query_json['dimensions'] = ["'NSR Data'[Beverage Category]"]
data.append(single_level(query_json))

##Zone
query_json['dimensions'] = ["'NSR Data'[L4 - Operations]"]
data.append(single_level(query_json))

##Zone+Channel
query_json['dimensions'] = ["'NSR Data'[L4 - Operations]","'NSR Data'[L3 - Market Type]"]
data.append(iterate_dynamic(query_json,2)[-1])

##Zone+Category
query_json['dimensions'] = ["'NSR Data'[L4 - Operations]","'NSR Data'[Beverage Category]"]
data.append(iterate_dynamic(query_json,2)[-1])

##Zone+Market+Category
query_json['dimensions'] = ["'NSR Data'[L4 - Operations]","'NSR Data'[L1 - Country]","'NSR Data'[L3 - Market Type]"]
data.append(iterate_dynamic(query_json,2)[-1])

##Zone+Market+Category
query_json['dimensions'] = ["'NSR Data'[L4 - Operations]","'NSR Data'[L1 - Country]","'NSR Data'[Beverage Category]"]
data.append(iterate_dynamic(query_json,2)[-1])


### Produce Narrative
narrative = iterateNarratives(data,prompt,summary_prompt)

Grouping: 'NSR Data'[L5 - Operating Unit]
Grouping: 'NSR Data'[L3 - Market Type]
Grouping: 'NSR Data'[Beverage Category]
Grouping: 'NSR Data'[L4 - Operations]
Grouping: ["'NSR Data'[L4 - Operations]"] Ranking Top 2: 'NSR Data'[L3 - Market Type]
Grouping: ["'NSR Data'[L4 - Operations]"] Ranking Top 2: 'NSR Data'[Beverage Category]
Grouping: ["'NSR Data'[L4 - Operations]"] Ranking Top 2: 'NSR Data'[L1 - Country]
Grouping: ["'NSR Data'[L4 - Operations]", "'NSR Data'[L1 - Country]"] Ranking Top 2: 'NSR Data'[L3 - Market Type]
Grouping: ["'NSR Data'[L4 - Operations]"] Ranking Top 2: 'NSR Data'[L1 - Country]
Grouping: ["'NSR Data'[L4 - Operations]", "'NSR Data'[L1 - Country]"] Ranking Top 2: 'NSR Data'[Beverage Category]


In [14]:
entry={
    "Report":dataset_name,
    "Table":f"{CurrentYear}_{LatestClosedMonth}_narrative",
    "Narrative":narrative,
    "Timestamp":timestamp
}
write_dict_to_excel("Narratives.xlsx", entry)

"The company’s performance this month in Europe indicates a strong upward trend compared to the previous year, with notable growth across several categories and channels exceeding the business plan targets. The overall 'Unit Cases % Growth' for Europe was at 3.0%, showing improvement from the last year's 1.7% growth, positioning the region well above the set business plan targets by 174.4 million cases.\n\nThe East region achieved a 'Unit Cases % Growth' of 5.2%, slightly lower than last year's 5.8%, but significantly surpassing the target with an absolute growth of 62.6 million cases. Meanwhile, the West region reversed its previous decline with a 'Unit Cases % Growth' of 1.8%, exceeding targets by 111.8 million cases.\n\nIn the category analysis, Sparkling Soft Drinks consistently drove performance with growth across most countries. In Europe overall, despite challenges in some segments, the category saw a growth of 2.9%, with a remarkable surplus against the target of 151.7 million 